# Ungraded Lab 1 - Company Data

This ungraded lab guides you through Laurence's interaction with the LLM in Module 3.

## Code from lecture Factory Patterns - Created by Laurence

The code below is in the lecture Factory Patterns, after the following prompt:

Enhance the following code to use the Factory gang-of-four pattern. Strictly follow the common conventions for the pattern. Start by explaining the conventions for the Factory pattern and why it makes sense to use it here. Then describe how the code modifications you made strictly follow the conventions of the pattern. 

There is also **One Self-paced exercise** in the end of this lab.

In [1]:
## Code for singleton, you must run this cell

import sqlite3
import pandas as pd
import numpy as np

# Singleton Pattern for Database Connection
class DatabaseConnection:
    _instance = None

    def __new__(cls, db_path):
        if cls._instance is None:
            cls._instance = super(DatabaseConnection, cls).__new__(cls)
            cls._instance.connection = sqlite3.connect(db_path)
        return cls._instance

    @staticmethod
    def get_connection():
        if DatabaseConnection._instance is None:
            raise Exception("DatabaseConnection has not been initialized. Call DatabaseConnection(db_path) first.")
        return DatabaseConnection._instance.connection

In [2]:
# Code for Factory Patterns

# Define the Bollinger Band width as a global variable
bollinger_width = 2

# Define the Window Size for Moving Average
window_size = 20

class Company:
    def __init__(self, company_id, ticker, name):
        self.company_id = company_id
        self.ticker = ticker
        self.name = name
        self.time_series = None
        self.high_bollinger = None
        self.low_bollinger = None
        self.moving_average = None
        self.grade = None

    def load_time_series(self, conn):
        query = '''
        SELECT date, value
        FROM TimeSeries
        WHERE company_id = ?
        ORDER BY date
        '''
        self.time_series = pd.read_sql_query(query, conn, params=(self.company_id,))
        self.time_series['date'] = pd.to_datetime(self.time_series['date'])

    def calculate_bollinger_bands(self):
        rolling_mean = self.time_series['value'].rolling(window_size).mean()
        rolling_std = self.time_series['value'].rolling(window_size).std()
        self.moving_average = rolling_mean
        self.high_bollinger = rolling_mean + (rolling_std * bollinger_width)
        self.low_bollinger = rolling_mean - (rolling_std * bollinger_width)

    def assign_grade(self, strategy):
        strategy.assign_grade(self)

    def display(self):
        print(f'Company: {self.name} ({self.ticker})')
        print(f'Grade: {self.grade}')
        print('Time Series Data:')
        print(self.time_series.tail())
        print('Moving Average:')
        print(self.moving_average.tail())
        print('High Bollinger Band:')
        print(self.high_bollinger.tail())
        print('Low Bollinger Band:')
        print(self.low_bollinger.tail())

class DomesticCompany(Company):
    def __init__(self, company_id, ticker, name):
        super().__init__(company_id, ticker, name)
        self.company_type = 'Domestic'

class ForeignCompany(Company):
    def __init__(self, company_id, ticker, name):
        super().__init__(company_id, ticker, name)
        self.company_type = 'Foreign'

class CompanyFactory:
    @staticmethod
    def get_company(identifier, conn):
        cursor = conn.cursor()

        if isinstance(identifier, str):
            query = 'SELECT id, ticker, name FROM companies WHERE ticker = ?'
            cursor.execute(query, (identifier,))
            row = cursor.fetchone()
            if row:
                return DomesticCompany(row[0], row[1], row[2])
        else:
            query = 'SELECT id, ticker, name FROM companies WHERE id = ?'
            cursor.execute(query, (identifier,))
            row = cursor.fetchone()
            if row:
                # If ticker is equal to ZZZZ, it's a foreign company
                if row[1] == 'ZZZZ':
                    return ForeignCompany(row[0], row[1], row[2])
                else:
                    return DomesticCompany(row[0], row[1], row[2])

class GradingStrategy:
    def assign_grade(self, company):
        raise NotImplementedError

class BollingerBandGradingStrategy(GradingStrategy):
    def assign_grade(self, company):
        latest_value = company.time_series['value'].iloc[-1]
        if latest_value > company.high_bollinger.iloc[-1]:
            company.grade = 'A'
        elif latest_value < company.low_bollinger.iloc[-1]:
            company.grade = 'C'
        else:
            company.grade = 'B'

# Adding error handling to the main code to catch any potential issues

# Example usage:
db_connection = DatabaseConnection('company_database.db')
conn = DatabaseConnection.get_connection()

# Get domestic company by ticker
try:
    domestic_company = CompanyFactory.get_company('AAPL', conn)
    if domestic_company:
        domestic_company.load_time_series(conn)
        domestic_company.calculate_bollinger_bands()
        grading_strategy = BollingerBandGradingStrategy()
        domestic_company.assign_grade(grading_strategy)
        domestic_company.display()
    else:
        print("Domestic company not found")
except Exception as e:
    print(f"Error processing domestic company: {e}")

# Get foreign company by ID
try:
    foreign_company = CompanyFactory.get_company(1001, conn)
    if foreign_company:
        foreign_company.load_time_series(conn)
        foreign_company.calculate_bollinger_bands()
        grading_strategy = BollingerBandGradingStrategy()
        foreign_company.assign_grade(grading_strategy)
        foreign_company.display()
    else:
        print("Foreign company not found")
except Exception as e:
    print(f"Error processing foreign company: {e}")

# Commenting out this line so you can run this cell more than once.
# conn.close()

Company: Apple Inc. (AAPL)
Grade: B
Time Series Data:
         date   value
95 2023-04-06  126.61
96 2023-04-07  167.88
97 2023-04-08  439.16
98 2023-04-09  384.38
99 2023-04-10  397.76
Moving Average:
95    271.0705
96    256.0275
97    257.4295
98    256.7370
99    254.4520
Name: value, dtype: float64
High Bollinger Band:
95    519.169173
96    489.728042
97    495.346003
98    493.002820
99    483.899798
Name: value, dtype: float64
Low Bollinger Band:
95    22.971827
96    22.326958
97    19.512997
98    20.471180
99    25.004202
Name: value, dtype: float64
Foreign company not found


## Self-paced Exercise

You now may use an LLM to do the following:

Implement a third kind of company - a **crypto currency**
- Traded 24 hours, so has no daily “closing” price

Question to think about with the LLM:

- Can you use the current TimeSeries database, or do you need a new one?
- Can you modify the code to calculate Bollinger bands, moving average etc. for these companies?

# Suggested solution (it is advised to try it yourself before checking it)

In [ ]:
class CryptoCompany(Company):
    def __init__(self, company_id, ticker, name):
        super().__init__(company_id, ticker, name)
        self.company_type = 'Crypto'

## Using a special ticker pattern for Crypto Currency - X. Redefining the CompanyFactory
class CompanyFactory:
    @staticmethod
    def get_company(identifier, conn):
        cursor = conn.cursor()

        if isinstance(identifier, str):
            query = 'SELECT id, ticker, name FROM companies WHERE ticker = ?'
            cursor.execute(query, (identifier,))
            row = cursor.fetchone()
            if row:
                # Check if the ticker indicates a cryptocurrency
                if row[1].startswith('X'):
                    return CryptoCompany(row[0], row[1], row[2])
                else:
                    return DomesticCompany(row[0], row[1], row[2])
        else:
            query = 'SELECT id, ticker, name FROM companies WHERE id = ?'
            cursor.execute(query, (identifier,))
            row = cursor.fetchone()
            if row:
                # If ticker is equal to ZZZZ, it's a foreign company
                if row[1] == 'ZZZZ':
                    return ForeignCompany(row[0], row[1], row[2])
                elif row[1].startswith('X'):
                    return CryptoCompany(row[0], row[1], row[2])
                else:
                    return DomesticCompany(row[0], row[1], row[2])

In [ ]:
# Inserting examples

def insert_crypto_companies_and_data():
    conn = DatabaseConnection.get_connection()  # Retrieve the database connection
    cursor = conn.cursor()

    # Insert cryptocurrency companies
    companies_insert_query = '''
    INSERT INTO companies (id, ticker, name) VALUES (?, ?, ?)
    '''
    crypto_companies_data = [
        (2001, 'XBTC', 'Bitcoin'),
        (2002, 'XETH', 'Ethereum')
    ]
    cursor.executemany(companies_insert_query, crypto_companies_data)

    # Insert time series data for the cryptocurrency companies
    time_series_insert_query = '''
    INSERT INTO TimeSeries (company_id, date, value) VALUES (?, ?, ?)
    '''
    time_series_data = [
        (2001, '2024-09-20', 43000.00),
        (2001, '2024-09-21', 43500.50),
        (2002, '2024-09-20', 3000.00),
        (2002, '2024-09-21', 3050.75)
    ]
    cursor.executemany(time_series_insert_query, time_series_data)

    conn.commit()  # Commit the transactions

# Example usage
db_connection = DatabaseConnection('company_database.db')  # Initialize the database connection
insert_crypto_companies_and_data()  # Insert the data

In [ ]:
## Retrieving the data

# Retrieve and display data for Bitcoin (XBTC)
try:
    crypto_company = CompanyFactory.get_company('XBTC', DatabaseConnection.get_connection())
    if crypto_company:
        crypto_company.load_time_series(DatabaseConnection.get_connection())
        crypto_company.calculate_bollinger_bands()
        grading_strategy = BollingerBandGradingStrategy()
        crypto_company.assign_grade(grading_strategy)
        crypto_company.display()
    else:
        print("Crypto company not found")
except Exception as e:
    print(f"Error processing crypto company: {e}")

In [ ]:
# Adding more data

import datetime

def insert_additional_crypto_data(conn, company_id, start_date, start_value):
    cursor = conn.cursor()
    
    # Generate 20 days of data
    for i in range(20):
        date = start_date + datetime.timedelta(days=i)
        value = start_value + (i * 100)  # Increment the value by 100 each day for simplicity
        
        cursor.execute('''
            INSERT INTO TimeSeries (company_id, date, value) VALUES (?, ?, ?)
        ''', (company_id, date.strftime('%Y-%m-%d'), value))
    
    conn.commit()

# Assuming you have a function to get your database connection
conn = DatabaseConnection.get_connection()

# Inserting additional data starting from the last known values
insert_additional_crypto_data(conn, 2001, datetime.date(2024, 9, 22), 43500.5)  # For Bitcoin
insert_additional_crypto_data(conn, 2002, datetime.date(2024, 9, 22), 3050.75)  # For Ethereum

In [ ]:
## Retrieving the data again

# Retrieve and display data for Bitcoin (XBTC)
try:
    crypto_company = CompanyFactory.get_company('XBTC', DatabaseConnection.get_connection())
    if crypto_company:
        crypto_company.load_time_series(DatabaseConnection.get_connection())
        crypto_company.calculate_bollinger_bands()
        grading_strategy = BollingerBandGradingStrategy()
        crypto_company.assign_grade(grading_strategy)
        crypto_company.display()
    else:
        print("Crypto company not found")
except Exception as e:
    print(f"Error processing crypto company: {e}")

# LLM prompts and outputs (it is advised to only check it after trying it yourself)